# This notebook is to plot all my credit card records.
    # The purpose is to keep track of my expenses and to know where all my money went...

In [24]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
#---- matplotlib, my old favorite...
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='ticks')
%matplotlib inline
#---- for Bokeh, a ploter
import bokeh.plotting as bp
from bokeh.layouts import gridplot, column
from bokeh.models import LinearAxis, DataRange1d, HoverTool, BoxSelectTool
TOOLS = ["pan,wheel_zoom,box_zoom,reset,crosshair, hover, box_select"]
WIDTH = 990
HEIGHT = 280
bp.output_notebook()
#---- for color, style print in iPython
from IPython.display import HTML, display
import datetime
import os
#---- for colorful print
from colorama import init
init(autoreset=True)
from colorama import Fore, Back, Style
#---- for statics
from scipy import stats

class ExpenseManager:
    def __init__(self):
        self.mitsuiVisaRecord = '../[SensitiveData/Expense/VISA_Mitsui/'
        self.JCBRecord = '../[SensitiveData/Expense/JCB/'
        self.Bank_SMBC = '../[SensitiveData/Bank_SMBC/'
        print("Initializing ExpenseManager instance")

    # read one csv file, Mitsui VISA format
    def readMitsuiVisaCSV(self, filePath):
        data = pd.read_csv(filePath, encoding='shift-JIS', header=None, usecols=[0,1,2], names=('Date', 'Category', 'Expense'))
        data = data[data['Date'] != '陳　飛宇　様']
        data = data[data['Date'].notnull()]
        # convert date, expense
        data['Date'] = pd.to_datetime(data['Date'])
        data['Expense'] = pd.to_numeric(data['Expense'])
        return data
    
    # read one csv file, JCB format
    def readJCB(self, filePath):
        data = pd.read_csv(filePath, encoding='shift-JIS', header=None, skiprows=[0,1,2,3,4,5],usecols=[2,3,4], names=('Date', 'Category', 'Expense'))
        data = data[data['Date'].notnull()]
        # convert date, expense
        data['Date'] = pd.to_datetime(data['Date'])
        #- replace ',' with ''
        data['Expense'] = data['Expense'].str.replace(',', '')
        data['Expense'] = pd.to_numeric(data['Expense'])
        return data
    
    # read one csv file, SMBC Bank account format
    def readBankSMBC(self, filePath):
        data = pd.read_csv(filePath, encoding='shift-JIS')
        return data

    # to convert the troublesome Japanese time format to YYYY-MM-DD
    def parse_heisei(self, date_string, sep='.'):
        y, m, d = date_string.split(sep)
        return datetime.date(year=1989 + int(y[1:]) -1, month=int(m), day=int(d))
    
    # read one folder, recurssively
    def readOneFolder(self, folderPath, whichRecord):
        for root, subdirs, files in os.walk(folderPath):
            #print('----------------')
            #print(Back.CYAN + Fore.RED + Style.BRIGHT + 
            #      'In Folder:'+root)
            for oneCsv in sorted(files):
                #print(Back.CYAN + oneCsv)
                filePath = folderPath+oneCsv
                #- VISA_Mitsui
                if whichRecord == 'MitsuiVISA':
                    data = self.readMitsuiVisaCSV(filePath)
                #- JCB, I don't usually use this card
                elif whichRecord == 'JCB':
                    data = self.readJCB(filePath)
                #- SMBC Bank account, this shall include all my money flow
                elif whichRecord == 'Bank_SMBC':
                    data = self.readBankSMBC(filePath)
                    data['年月日（和暦）'] = data['年月日（和暦）'].apply(self.parse_heisei)
                    
                try:
                    allData
                except NameError:   # allData not exist
                    allData = data
                else:   # allData exist
                    # concatenate 2 pandas dataframes
                    allData = pd.concat([allData, data])
        allData.index = range(len(allData))
        return allData
    
    # choose the data within [StartDate, EndDate], return a mask
    def chooseDateBasedonDate(self, dataframe, startDate, endDate):
        mask = (startDate < dataframe['Date']) & (dataframe['Date'] < endDate)
        return mask
    
em = ExpenseManager()

Loading BokehJS ...

Initializing ExpenseManager instance


In [12]:
mitsuiVisa = em.readOneFolder(em.mitsuiVisaRecord, 'MitsuiVISA')
JCB = em.readOneFolder(em.JCBRecord, 'JCB')

In [13]:
print(mitsuiVisa)

           Date                                  Category   Expense
0    2014-12-14                                      ローソン     529.0
1    2014-12-17                          サークルＫコマハ゛４チヨウメテン     551.0
2    2014-12-18                              ＡＭＡＺＯＮ．ＣＯ．ＪＰ    4920.0
3    2014-12-18                                      ローソン     113.0
4    2014-12-19                               サミットストア　成城店    2170.0
5    2014-12-20                               サミットストア　成城店    2989.0
6    2014-12-22                               サミットストア　成城店    1262.0
7    2014-12-22                          サークルＫコマハ゛４チヨウメテン     311.0
8    2014-12-24                               サミットストア　成城店    1841.0
9    2014-12-27                               サミットストア　成城店    1302.0
10   2014-12-28                               サミットストア　成城店     957.0
11   2014-12-28                               サミットストア　成城店    2392.0
12   2014-12-29                               サミットストア　成城店    1474.0
13   2014-12-30                               サミ

1185 2017-12-31                        アメリカンドラッグ山ノ内平穏店／ｉＤ     941.0


In [14]:
print(JCB)

          Date                             Category  Expense
0   2016-06-02                     フアミリ−マ−トゾウシキエキマエ      271
1   2016-06-03                          菜食健美　よこはま農園    35000
2   2016-06-03                     フアミリ−マ−トゾウシキエキマエ     1014
3   2016-06-04                                  ダイス      386
4   2016-06-05                         プロントコーポレーション     5820
5   2016-06-05                          スターバックスコーヒー      550
6   2016-06-05                        ＬＡ　ＣＩＴＴＡＤＥＬＬＡ     2851
7   2016-06-06                     フアミリ−マ−トゾウシキエキマエ      366
8   2016-06-10                   ＴＯＫＹＯ　ＴＯＷＥＲ　ＧＡＬＡＸＹ      594
9   2016-06-10                          東京タワーチケット売場      700
10  2016-06-11                             東京スカイツリー     2060
11  2016-06-11                             東京スカイツリー     1250
12  2016-06-11                             東京スカイツリー     1030
13  2016-06-11                             東京スカイツリー      515
14  2016-06-11                             東京スカイツリー      480
15  2016-06-12          

371 2017-08-05                              UDEMYEU     3600


In [15]:
dataVisa = mitsuiVisa.loc[em.chooseDateBasedonDate(mitsuiVisa, '2017-08', '2017-09')]
print(dataVisa)

           Date                               Category  Expense
953  2017-08-03                               ファミリーマート    100.0
954  2017-08-04                          セブン−イレブン・ジャパン    585.0
955  2017-08-05                   スタ−バックスコ−ヒ−　有楽町ビル１階店    432.0
956  2017-08-06                          セブン−イレブン・ジャパン    229.0
957  2017-08-06                                  ジョナサン   5007.0
958  2017-08-07                               ファミリーマート    266.0
959  2017-08-08                          セブン−イレブン・ジャパン    591.0
960  2017-08-08                               オカダヤモア―ズ    345.0
961  2017-08-09                          セブン−イレブン・ジャパン    583.0
962  2017-08-09                                   マルエツ   2413.0
963  2017-08-10                                 六本木ヒルズ    432.0
964  2017-08-10                   ＡＮＡインターネットチケットレスサービス  40680.0
965  2017-08-10                      トヨタレンタリース新札幌Ｗｅｂ決済  39420.0
966  2017-08-10                               ての字　大手町店   6230.0
967  2017-08-10                         

In [16]:
dataJCB = JCB.loc[em.chooseDateBasedonDate(JCB, '2017-08', '2017-09')]
print(dataJCB)

          Date              Category  Expense
369 2017-08-04  イモトのＷｉ−Ｆｉ　エクスコムグロ−バル     4320
370 2017-08-05              マルエツ〔食品〕     1734
371 2017-08-05               UDEMYEU     3600


In [17]:
print(dataVisa['Expense'].sum())
print(dataJCB['Expense'].sum())

268372.0
9654


In [18]:
#--- figure plot example
# 1. construct pandas dataframe
dataPlot = pd.DataFrame({
    'Cards': ['Visa', 'JCB'],
    'Month': ['2017-08', '2017-08'],
    'Expense': [dataVisa['Expense'].sum(), dataJCB['Expense'].sum()]
})
fig = Bar(dataPlot, label='Month', values='Expense', group='Cards', legend='top_right', tools=TOOLS, toolbar_location="above")
fig.legend.background_fill_alpha = 0.3
fig.legend.click_policy="hide"
bp.show(fig)


NameError: name 'Bar' is not defined

In [25]:
# 选出Dataframe的“お取り扱い内容”column中含有“ロト”的row，然后对“お引出し”column做sum

Bank_SMBC = em.readOneFolder(em.Bank_SMBC, 'Bank_SMBC')
print(Bank_SMBC)
print('----------------------------------')

# find all rows whose 'お取り扱い内容' cloumn contain 'ロト'
LotoExpense = Bank_SMBC.loc[Bank_SMBC['お取り扱い内容'].str.contains('ロト')]
print(LotoExpense)

print(LotoExpense['お引出し'].sum())

       年月日（和暦）      お引出し      お預入れ                  お取り扱い内容       残高
0   2017-12-01       NaN  663973.0           賞与振込　ﾊﾟﾅｿﾆﾂｸ(ｶ  5600170
1   2017-12-01     600.0       NaN         第２４１回　ロト７　ﾀｶﾗｸｼﾞ  5599570
2   2017-12-01     400.0       NaN        第１２３２回　ロト６　ﾀｶﾗｸｼﾞ  5599170
3   2017-12-07     400.0       NaN        第１２３３回　ロト６　ﾀｶﾗｸｼﾞ  5598770
4   2017-12-07     600.0       NaN         第２４２回　ロト７　ﾀｶﾗｸｼﾞ  5598170
5   2017-12-10   30000.0       NaN                カード　(624)  5568170
6   2017-12-10     108.0       NaN                   カード手数料  5568062
7   2017-12-11       NaN    4336.0  振込　ﾊﾟﾅｿﾆﾂｸ(ｶ)ｼﾕｳﾁﾕｳﾌﾘｺﾐ  5572398
8   2017-12-13     600.0       NaN         第２４３回　ロト７　ﾀｶﾗｸｼﾞ  5571798
9   2017-12-13     400.0       NaN        第１２３５回　ロト６　ﾀｶﾗｸｼﾞ  5571398
10  2017-12-20     400.0       NaN        第１２３７回　ロト６　ﾀｶﾗｸｼﾞ  5570998
11  2017-12-20     600.0       NaN         第２４４回　ロト７　ﾀｶﾗｸｼﾞ  5570398
12  2017-12-25       NaN  177116.0           給料振込　ﾊﾟﾅｿﾆﾂｸ(ｶ  5747514
13  2017-12-25       NaN   53400.0